In [2]:
import polars as pl
import torch
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors
import numpy as np
from amaretto_mlp import *
from utils_b import *
from autoencoder import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

# 1. Data & Models Setup

## 1.1. Load data & model

In [3]:
pl.Config.set_tbl_cols(132) 

polars.config.Config

In [4]:
df = pl.read_parquet("amaretto_transformed_scaled.pq")
df_train, df_val, df_test = split_data(df)
columns_to_drop = ['id', 'Anomaly', 'Anomaly_bin', 'Originator', 'datetime']

target_column = 'Anomaly'
y_train = df_train.select(target_column).to_numpy().flatten()
df_train = df_train.drop(columns_to_drop)
X_train = df_train.to_numpy()
y_val = df_val.select(target_column).to_numpy().flatten()
X_val = df_val.drop(columns_to_drop).to_numpy()
y_test = df_test.select(target_column).to_numpy().flatten()
X_test = df_test.drop(columns_to_drop).to_numpy()

train_dataset = AmarettoDataset(X_train, y_train)
val_dataset = AmarettoDataset(X_val, y_val)
test_dataset = AmarettoDataset(X_test, y_test)

print(X_train.shape)
print(X_test.shape)

(24451934, 125)
(2765755, 125)


In [5]:
train_loader = DataLoader(train_dataset, batch_size=256, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=256, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=256, num_workers=2)
input_dim = X_train.shape[1]
df_train, df_val, df_test = split_data(df)
print(len(train_loader))

95516


In [6]:
model = CombinedModel(input_dim=X_train.shape[1], hidden_layers=[64, 32], latent_dim=16, num_classes=6)
model.load_state_dict(torch.load("model_combined.pth"))
print(model)

CombinedModel(
  (encoder): Encoder(
    (network): Sequential(
      (0): Linear(in_features=125, out_features=64, bias=True)
      (1): Tanh()
      (2): Linear(in_features=64, out_features=32, bias=True)
      (3): Tanh()
      (4): Linear(in_features=32, out_features=16, bias=True)
    )
  )
  (decoder): Decoder(
    (network): Sequential(
      (0): Linear(in_features=16, out_features=32, bias=True)
      (1): Tanh()
      (2): Linear(in_features=32, out_features=64, bias=True)
      (3): Tanh()
      (4): Linear(in_features=64, out_features=125, bias=True)
    )
  )
  (classifier): Classifier(
    (network): Sequential(
      (0): Linear(in_features=16, out_features=16, bias=True)
      (1): ReLU()
      (2): Linear(in_features=16, out_features=6, bias=True)
    )
  )
)


## 1.2. Obtain laten space and predictions for train and test data

In [7]:
model.eval()
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
with torch.no_grad():
    _, y_pred_train, z_train = model(X_train_tensor)
    y_pred_train = torch.argmax(y_pred_train, dim=1)
y_pred_train = y_pred_train.cpu().detach().numpy()
print(y_pred_train.shape)
z_train = z_train.cpu().numpy()
print(z_train.shape)

(24451934,)
(24451934, 16)


In [8]:
model.eval()
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
with torch.no_grad():
    _, y_pred_test, z_test = model(X_test_tensor)
    y_pred_test = torch.argmax(y_pred_test, dim=1)
y_pred_test = y_pred_test.cpu().detach().numpy()
print(y_pred_test.shape)
z_test = z_test.cpu().numpy()
print(z_test.shape)

(2765755,)
(2765755, 16)


In [9]:
df_test = df_test.with_columns(pl.Series("Model Prediction", y_pred_test))
df_test = df_test.select(["Model Prediction", *df_test.columns[:-1]])  
df_test = df_test.with_row_index(name="index")
df_test

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0,27025095,2019-03-18 00:00:01,"""Client_023""",0,0,0.507711,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,18,0,0,false,1.771247,1.252211,1.039728,0.94558,-2.511988,1.660603,-0.284955,-0.112912,0.0,0.0,-1.390533,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,-1.175981,-1.290214,-0.000167,0.000056,0.71708,1.365456,0.513469,-1.287407,0.279149,3.620574,0.147873,1.40623,-284.418707,-594.740725,-701.775893,-0.891973,-40.751886,-33.504513,-40.

In [10]:
df_train = df_train.with_columns(pl.Series("Model Prediction", y_pred_train))
df_train = df_train.select(["Model Prediction", *df_train.columns[:-1]])  
df_train = df_train.with_row_index(name="index")
df_train

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,2,151925,2019-01-01 00:00:03,"""Client_259""",0,0,0.392679,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,2,1,0,0,false,-5.176871,-4.005506,-3.292225,-2.835853,-2.511988,-0.669069,-0.284955,-0.112912,0.0,0.0,-1.390533,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,-0.000079,-0.000208,-0.000167,0.000056,-0.811648,0.879612,0.087694,1.476385,0.279149,3.620574,0.147873,1.40623,-284.418707,-594.740725,-701.775893,-1850.737809,-40.751886,-33.5045

## 1.3. Evaluation results

In [11]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00   2759074
           1       0.24      0.93      0.38       576
           2       0.28      0.99      0.43       257
           3       0.69      0.92      0.79      1175
           4       0.48      0.87      0.62      2228
           5       0.46      0.39      0.42      2445

    accuracy                           1.00   2765755
   macro avg       0.53      0.85      0.61   2765755
weighted avg       1.00      1.00      1.00   2765755



## 1.4. Define k-NN Search Method


In [12]:
knn = NearestNeighbors(n_neighbors=9, algorithm='auto', metric='euclidean')
knn.fit(z_train)

NearestNeighbors(metric='euclidean', n_neighbors=9)

# 2. Analysis for Each Anomaly Type

In [13]:
mask_correct_preds = (y_test[y_test != 0] == y_pred_test[y_test != 0])
mask_wrong_preds = (y_test[y_test != 0] != y_pred_test[y_test != 0])
print(mask_correct_preds.sum())
print(mask_wrong_preds.sum())

4768
1913


In [14]:
df_correct_preds = df_test.filter((pl.col("Anomaly") != 0))
df_correct_preds = df_correct_preds.filter(mask_correct_preds)
df_wrong_preds = df_test.filter((pl.col("Anomaly") != 0))
df_wrong_preds = df_wrong_preds.filter(mask_wrong_preds)
print(df_correct_preds.height)
print(df_wrong_preds.height)

4768
1913


## 2.1. Small but highly frequent transactions generated within a short timeframe:
A pattern that contains multiple transactions below applicable reporting thresholds.  
Selected indexes from testing set: 38329, 62415

### 2.1.1. Correct prediction:

In [15]:
x_38329 = torch.tensor(X_test[38329], dtype=torch.float32)
x_38329[:6]

tensor([-1.4487,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000])

In [16]:
model.encoder.eval()
test_transaction_latent = model.encoder(x_38329)
test_transaction_latent = test_transaction_latent.cpu().detach().numpy()
test_transaction_latent = test_transaction_latent.reshape(1, -1)
test_transaction_latent.shape

(1, 16)

In [17]:
k = 9
distances, indices = knn.kneighbors(test_transaction_latent, n_neighbors=k)
print(indices)
print(distances)

[[15331650  5746304  4658090 14831910  4618398  4653400  4747816  5627063
   6337860]]
[[0.76601273 1.0838995  1.08571053 1.10980785 1.14570367 1.15599835
  1.1784091  1.19861209 1.22553897]]


In [18]:
df_correct_preds.filter(pl.col("index") == 38329)

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
38329,1,26944892,2019-03-18 08:01:16,"""Client_389""",1,1,-1.448741,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,18,8,1,false,2.054952,1.690853,1.400966,1.251966,1.166733,-0.669069,-0.284955,-0.112912,0.0,0.0,1.037127,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,0.549258,-0.000208,0.646075,0.000056,0.71708,1.365456,0.513469,-1.287407,-1.364252,0.64142,0.000044,1.413977,0.344561,0.199322,0.2127,-0.933059,0.123479,-2.25052,-3.695437,0.203

In [19]:
df_train[indices.flatten().tolist()]

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
15331650,1,15308548,2019-02-13 08:06:31,"""Client_255""",0,0,-1.533725,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,3,13,8,6,false,0.438458,0.947234,0.879019,1.062288,1.255667,-0.669069,-0.284955,-0.112912,0.0,0.0,0.989963,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,0.495508,-0.000208,-1.119112,0.000056,-1.189212,-0.212069,-0.529405,-1.212206,-1.364252,0.64142,-0.707075,1.224512,0.250799,0.362332,0.482231,0.425381,0.550979,-1.248944,0.47

In [20]:
print(z_train[indices.flatten().tolist()][1:3])

[[-1.9108541  -0.2559423  -0.49183255 -3.5275347  -0.878192   -0.5947329
  -0.40787393  2.1667094  -2.2090037  -0.6234193  -2.2077599   1.5936036
   0.5121195   0.7161606  -0.76077384 -3.0988145 ]
 [-2.4169302  -0.69858044 -0.21707273 -3.929904   -0.58289784 -0.2279321
  -0.92739505  2.019814   -1.7268871  -0.6007668  -1.991197    0.9367852
   0.8320989   0.69550276 -0.8708064  -2.4754653 ]]


In [21]:
print(z_test[38329])

[-1.8543928  -0.39676404 -0.3398804  -3.7538128  -0.46145567 -0.41433507
 -1.0469179   2.279829   -1.8658466  -0.6115927  -2.1304078   1.3584336
  0.635863    0.39090097 -0.5040114  -2.8091183 ]


### 2.1.2. Incorrect prediction:

In [22]:
df_test.filter((pl.col("Anomaly") == 1) & (pl.col("Model Prediction") != 1))

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
8357,2,27181276,2019-03-18 07:07:36,"""Client_126""",1,1,-1.374993,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,18,7,7,true,2.040487,1.445817,1.199385,1.077004,1.010305,-0.669069,-0.284955,-0.112912,0.0,0.0,-1.390533,-0.917766,-0.698787,-0.576665,-0.484611,2.410951,-0.183119,-0.071133,0.0,0.0,-1.058838,-0.000208,-0.000167,-2.606994,0.71708,1.365456,0.513469,-1.287407,-1.409974,1.270989,-0.831225,1.143892,-1.830154,-3.55436,-4.216388,-0.910444,-3.529966,-4.415957,-6.225

In [23]:
idx = 62415

In [24]:
x = torch.tensor(X_test[idx], dtype=torch.float32)
x[:6]

tensor([-1.3394,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000])

In [25]:
model.encoder.eval()
test_transaction_latent = model.encoder(x)
test_transaction_latent = test_transaction_latent.cpu().detach().numpy()
test_transaction_latent = test_transaction_latent.reshape(1, -1)
test_transaction_latent.shape

(1, 16)

In [26]:
distances, indices = knn.kneighbors(test_transaction_latent, n_neighbors=k)
print(indices)
print(distances)

[[ 9632490  2099470  2068467  2148546  2064604 11997058  2104307  6825048
   9632670]]
[[0.82537657 0.87800348 0.88829303 0.89292318 0.90112567 0.91577238
  0.92514712 0.94818842 0.95283419]]


In [27]:
df_wrong_preds.filter(pl.col("index") == idx)

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
62415,0,27375822,2019-03-18 08:44:02,"""Client_269""",1,1,-1.339388,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,18,8,44,false,1.861706,1.948089,1.612532,1.436201,1.332469,-0.669069,-0.284955,-0.112912,0.0,0.0,1.037161,-0.917766,-0.698787,-0.576665,-0.484611,2.411036,-0.183119,-0.071133,0.0,0.0,-0.975575,-0.000208,0.663665,-2.56952,0.71708,1.365456,0.513469,-1.287407,-1.364252,0.64142,1.38338,-0.294236,0.252691,0.315733,0.350057,-0.914695,0.122604,-1.588695,-2.922173,0

In [28]:
df_train[indices.flatten().tolist()]

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
9632490,0,9754351,2019-01-28 07:59:06,"""Client_320""",0,0,-1.595541,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,28,7,59,false,-0.027058,0.104074,0.099028,0.130384,0.199612,1.542487,-0.284955,-0.112912,0.0,0.0,0.962846,-0.917766,-0.698787,-0.576665,-0.484611,2.308156,-0.183119,-0.071133,0.0,0.0,-1.082775,0.855812,0.521247,-2.659604,0.71708,1.365456,1.118543,1.036483,-1.409974,1.270989,0.294081,1.383074,-1.195856,-2.104188,-2.505285,0.065946,0.352222,-2.07602,-3.491555

In [29]:
print(z_train[indices.flatten().tolist()][0])

[ 0.05787796  0.07456173 -0.1332953  -1.2503579   0.0344611   1.6695037
 -0.49060786  0.8539669   0.7130349   0.10567185 -0.18471116 -0.5933391
  0.2587705  -0.7097423   1.3995408   1.0635419 ]


In [30]:
print(z_test[idx])

[ 0.21056983 -0.12642956 -0.43496287 -1.5170878  -0.02542047  1.6954181
 -0.78687716  0.998266    0.7091562  -0.2391444  -0.25096908 -0.6613333
 -0.11959938 -0.7973846   1.4859515   1.3008024 ]


## 2.2. Transactions with rounded normalized amounts bought or sold within an account:
It is unusual for transactions in capital markets to have rounded amounts (unless they occur in markets where foreign exchange conversion causes rounding errors).  
Selected indexes from testing set: 2762491, 1154

### 2.2.1. Correct prediction:

In [31]:
idx = 2762491
x = torch.tensor(X_test[idx], dtype=torch.float32)
x[:6]

tensor([-0.3213,  0.0000,  1.0000,  0.0000,  1.0000,  0.0000])

In [32]:
model.encoder.eval()
test_transaction_latent = model.encoder(x)
test_transaction_latent = test_transaction_latent.cpu().detach().numpy()
test_transaction_latent = test_transaction_latent.reshape(1, -1)
test_transaction_latent.shape

(1, 16)

In [33]:
distances, indices = knn.kneighbors(test_transaction_latent, n_neighbors=k)
print(indices)
print(distances)

[[ 6554950 20753585  8001378  7524847  2028776 20751676 20591714  7112699
   7490872]]
[[0.76053721 0.89949995 0.94712424 0.96675688 1.00107718 1.0123415
  1.04162109 1.05858374 1.07525563]]


In [34]:
df_correct_preds.filter(pl.col("index") == idx)

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2762491,2,29131332,2019-03-25 20:57:41,"""Client_066""",2,1,-0.321324,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,25,20,57,true,2.723018,1.960465,1.622718,1.445065,1.340447,1.625231,-0.284955,-0.112912,0.0,0.0,-1.390533,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,0.870832,-1.839677,-0.000167,0.000056,0.71708,1.365456,1.493561,0.276227,1.227968,1.900558,0.575268,1.291715,-0.409452,-0.767024,-0.903726,-0.809743,-0.019948,1.159269,0.5462

In [35]:
df_train[indices.flatten().tolist()]

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
6554950,2,6300147,2019-01-17 14:54:01,"""Client_212""",2,1,-0.190208,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,4,17,14,54,true,1.796033,1.26195,1.048186,0.944911,0.891121,1.529763,3.440718,-0.112912,0.0,0.0,0.868168,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,-0.985445,-2.224226,-1.083662,0.000056,-0.1313,-1.087529,0.231583,-1.345012,0.279149,-1.078596,0.946355,1.050736,0.167621,0.567472,0.523855,-0.001505,0.290258,0.533247,0.532868,0

In [36]:
print(z_train[indices.flatten().tolist()][0])

[-2.2053099   0.24727105  1.1828415  -3.7803001  -0.5508146  -1.5143605
  0.01208356  1.747623   -0.6920035  -1.3358337  -0.90480244 -0.19344726
  0.9980798   1.3941483   0.5763866  -3.5103214 ]


In [37]:
print(z_test[idx])

[-1.966109    0.4253183   1.1670809  -3.9706533  -0.47598058 -1.5607606
 -0.22768898  1.6013286  -0.84036523 -1.244536   -0.68707526 -0.23627256
  0.6007144   1.1737615   0.40720263 -3.2829642 ]


### 2.2.2. Inorrect prediction:

In [38]:
idx = 1154
x = torch.tensor(X_test[idx], dtype=torch.float32)
x[:6]

tensor([0.4580, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000])

In [39]:
model.encoder.eval()
test_transaction_latent = model.encoder(x)
test_transaction_latent = test_transaction_latent.cpu().detach().numpy()
test_transaction_latent = test_transaction_latent.reshape(1, -1)
test_transaction_latent.shape

(1, 16)

In [40]:
distances, indices = knn.kneighbors(test_transaction_latent, n_neighbors=k)
print(indices)
print(distances)

[[19319741  4603582  4603425  7115916  4603372  7115920  4603570  4603567
  19320089]]
[[0.42078573 0.55959392 0.56648523 0.63580823 0.69816321 0.74333048
  0.84333616 0.88101697 0.88444638]]


In [41]:
df_wrong_preds.filter(pl.col("index") == idx)

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1154,2,27350159,2019-03-18 02:02:56,"""Client_066""",3,1,0.458002,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,18,2,2,false,2.693317,1.938037,1.604243,1.428987,1.32599,1.620093,-0.284955,-0.112912,0.0,0.0,1.036347,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,1.038604,1.832056,1.05927,0.000056,0.71708,1.365456,0.513469,-1.287407,-0.415434,3.620574,-0.147784,1.40623,-7.97239,-2.151224,-2.560785,-0.885804,-7.797523,-7.049038,-9.301945,0.20

In [42]:
df_train[indices.flatten().tolist()]

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
19319741,2,19373735,2019-02-25 02:01:33,"""Client_066""",3,1,0.488824,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,25,2,1,false,2.613161,1.877475,1.554345,1.385519,1.286867,-0.669069,-0.284955,-0.112912,0.0,0.0,1.009864,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,0.803864,-0.000208,1.208517,0.000056,0.71708,1.365456,1.199668,0.943877,-0.415434,3.620574,0.000044,1.413977,-9.279982,-4.438574,-5.259701,0.31851,-7.72811,-6.959737,-9.197607

In [43]:
print(z_train[indices.flatten().tolist()][0])

[-2.580623    0.40725732 -0.08335793 -3.7024746   0.39335    -0.5646753
 -0.17899056  1.43648    -2.6491487  -0.87560505 -2.7263784   0.29130527
  1.6478683  -0.9668075  -1.541463   -2.2553492 ]


In [44]:
print(z_test[idx])

[-2.6692252   0.39351767 -0.11413684 -3.536933    0.47446314 -0.46035907
 -0.12506744  1.186397   -2.5895786  -0.8051623  -2.7098064   0.29945144
  1.5998503  -1.0477828  -1.6022947  -2.064708  ]


## 2.3. Security bought or sold at an unusual time:
It is unusual for clients to trade specific securities outside of a specific timeframe (for example, outside of the opening and closing times of a stock exchange).  
Selected indexes from testing set: 1346, 1781682

### 2.3.1. Correct prediction:

In [45]:
df_correct_preds.filter((pl.col("Anomaly") == 3))

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1174,3,27366041,2019-03-18 02:04:55,"""Client_126""",3,1,0.472797,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,18,2,4,false,2.003046,1.436655,1.192007,1.070899,1.003866,-0.669069,-0.284955,-0.112912,0.0,0.0,1.036352,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,0.914111,-0.000208,-1.019988,0.000056,0.71708,1.365456,0.513469,-1.287407,-0.415434,3.620574,-0.43698,1.344762,-8.217688,-2.861691,-3.399087,-0.885898,-7.804154,-7.04613,-9.298548

In [46]:
idx = 1346
x = torch.tensor(X_test[idx], dtype=torch.float32)
x[:6]

tensor([0.4183, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000])

In [47]:
model.encoder.eval()
test_transaction_latent = model.encoder(x)
test_transaction_latent = test_transaction_latent.cpu().detach().numpy()
test_transaction_latent = test_transaction_latent.reshape(1, -1)
test_transaction_latent.shape

(1, 16)

In [48]:
distances, indices = knn.kneighbors(test_transaction_latent, n_neighbors=k)
print(indices)
print(distances)

[[4603524 4603558 4603502 4603495 4603610 4603675 4603653 4603446 9588528]]
[[0.46142513 0.46398443 0.48677099 0.5048399  0.50544786 0.50720632
  0.5305391  0.53996813 0.54339254]]


In [49]:
df_correct_preds.filter(pl.col("index") == idx)

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1346,3,27246331,2019-03-18 02:20:04,"""Client_066""",3,1,0.418297,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,18,2,20,false,2.693365,1.938079,1.604288,1.429031,1.326017,1.660752,-0.284955,-0.112912,0.0,0.0,1.036389,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,0.954385,1.674953,0.913214,0.000056,0.71708,1.365456,0.513469,-1.287407,-0.415434,3.620574,-1.291928,-0.575412,-7.119672,-5.13998,-6.087311,-0.886671,-7.616917,-6.926563,-9.158848

In [50]:
df_train[indices.flatten().tolist()]

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
4603524,2,5028796,2019-01-14 02:11:03,"""Client_066""",3,1,0.417803,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,14,2,11,false,2.239508,1.595348,1.321938,1.183096,1.104706,-0.669069,-0.284955,-0.112912,0.0,0.0,-1.390533,-0.917766,-0.698787,-0.576665,-0.484611,2.248287,-0.183119,-0.071133,0.0,0.0,-0.951451,-0.000208,-0.000167,-2.406382,0.71708,1.365456,-0.602535,-1.174554,-0.415434,3.620574,-1.224723,0.706884,-9.139821,-8.850709,-10.465718,-0.690316,-7.349266,-6.82381,-

In [53]:
print(z_train[indices.flatten().tolist()][:2])

[[-2.943015    0.49808413 -0.44908285 -3.8868234   0.09194685 -0.69445544
  -0.33399194  1.3613456  -2.6805503  -0.7521832  -2.4990182   0.14477342
   1.4030517  -0.54363847 -1.9090625  -1.6402696 ]
 [-2.9167001   0.49047643 -0.50252855 -3.907482    0.19890735 -0.50764227
  -0.42853248  1.4338256  -2.6403399  -0.725737   -2.3732367   0.10414317
   1.394406   -0.4753898  -1.7909226  -1.5564725 ]]


In [52]:
print(z_test[idx])

[-3.1553469   0.6383329  -0.4086944  -3.9656613   0.2972519  -0.7436959
 -0.51217353  1.3206539  -2.5542321  -0.7695734  -2.3378134   0.03436419
  1.4448581  -0.5661607  -1.8366312  -1.6601394 ]


### 2.3.2. Incorrect prediction:

In [54]:
df_wrong_preds.filter((pl.col("Model Prediction") == 3))

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
355240,3,27065459,2019-03-18 17:30:02,"""Client_066""",5,1,-1.675861,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,18,17,30,true,2.695845,1.939912,1.605804,1.430336,1.327207,-0.669069,-0.284955,-0.112912,0.0,0.0,0.34641,0.723354,1.050612,1.336038,1.589677,2.411504,-0.183119,-0.071133,0.0,0.0,-0.960249,-0.000208,-0.349265,-2.607035,0.71708,1.365456,0.513469,-1.287407,1.093918,0.054755,-0.147784,-1.406651,-1.137572,-0.546782,-0.671237,-1.095911,-0.003043,0.723069,-0.20668

In [55]:
idx = 1781682
x = torch.tensor(X_test[idx], dtype=torch.float32)
x[:6]

tensor([-1.6971,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000])

In [56]:
model.encoder.eval()
test_transaction_latent = model.encoder(x)
test_transaction_latent = test_transaction_latent.cpu().detach().numpy()
test_transaction_latent = test_transaction_latent.reshape(1, -1)
test_transaction_latent.shape

(1, 16)

In [57]:
distances, indices = knn.kneighbors(test_transaction_latent, n_neighbors=k)
print(indices)
print(distances)

[[15677286 18171622 16848173 15845253 22827709 18172765 17306167  6746998
  20311927]]
[[1.36728764 1.44455564 1.46582866 1.46735299 1.54738748 1.58399832
  1.58922672 1.65531695 1.67415595]]


In [58]:
df_wrong_preds.filter(pl.col("index") == idx)

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1781682,3,28518201,2019-03-21 20:09:59,"""Client_212""",4,1,-1.697112,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,4,21,20,9,true,2.29801,1.639747,1.35928,1.216279,1.13909,-0.669069,-0.284955,-0.112912,0.0,0.0,0.457708,0.811679,1.132911,1.990487,2.300806,2.415823,-0.183119,-0.071133,0.0,0.0,-1.043263,-0.000208,-1.261312,-2.516204,-0.1313,-1.087529,1.212389,-0.80366,1.227968,1.900558,-1.05094,0.946066,1.330779,-0.379375,-0.455153,-0.45867,0.057545,0.866354,0.199839,-0.022

In [59]:
df_train[indices.flatten().tolist()]

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
15677286,4,15525141,2019-02-13 16:11:24,"""Client_212""",0,0,0.491088,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,3,13,16,11,false,2.105013,1.494872,1.240298,1.113313,1.041979,-0.669069,-0.284955,-0.112912,0.0,0.0,0.987421,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,0.960076,-0.000208,-1.687497,0.000056,-1.189212,-0.212069,-0.529405,-1.212206,0.880676,-0.449026,-1.224723,0.706884,0.757448,-0.014887,-0.052468,0.313411,0.594052,0.895141,0

In [60]:
print(z_train[indices.flatten().tolist()][:2])

[[-2.0570624   0.17379062 -0.8519243  -3.6490903  -0.49881846 -0.87352675
  -0.5234573   0.3419947  -2.4500933  -1.1359335  -1.8786758   1.4867889
  -0.42023656  1.1299585  -1.3835087  -2.1449351 ]
 [-2.051531    0.06298916 -0.8561177  -3.639601   -0.7383856  -1.0891514
   0.06118202  0.4935416  -2.7967646  -1.1235758  -2.0418215   1.4637522
  -0.43159977  1.6191574  -1.4891049  -2.096597  ]]


In [61]:
print(z_test[idx])

[-1.9576206   0.48322308 -0.4118706  -3.5923517  -0.25153545 -1.6087607
 -0.08688624  0.2753136  -2.8073256  -1.1732817  -2.0877256   1.5980335
 -0.3997041   1.0095682  -0.9852133  -2.786474  ]


## 2.4. Large asset withdrawal:
A sudden spike in transaction amount withdrawn from an account or transferred out, which deviates from the previous transactional activity and is absent of any commercial rationale or related corporate action event.  
Selected indexes from testing set: 4495, 2761298

### 2.4.1. Correct prediction:

In [62]:
df_correct_preds.filter((pl.col("Anomaly") == 4))

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
4113,4,27012282,2019-03-18 07:00:02,"""Client_066""",4,1,0.528073,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,18,7,0,false,2.694187,1.938685,1.604796,1.429458,1.326411,-0.669069,-0.284955,-0.112912,0.0,0.0,0.839761,1.145717,1.465387,1.72366,1.990764,-0.433523,-0.183119,-0.071133,0.0,0.0,-1.131476,-0.000208,-1.242024,0.000056,0.71708,1.365456,0.513469,-1.287407,-1.409974,1.270989,0.147873,1.40623,-21.190487,-6.315033,-7.473795,-0.893325,-8.810725,-5.451748,-7.435699,0.

In [63]:
idx = 4495
x = torch.tensor(X_test[idx], dtype=torch.float32)
x[:6]

tensor([1.3561, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000])

In [64]:
model.encoder.eval()
test_transaction_latent = model.encoder(x)
test_transaction_latent = test_transaction_latent.cpu().detach().numpy()
test_transaction_latent = test_transaction_latent.reshape(1, -1)
test_transaction_latent.shape

(1, 16)

In [65]:
distances, indices = knn.kneighbors(test_transaction_latent, n_neighbors=k)
print(indices)
print(distances)

[[16917465 16920035 16918342 14233026 14238390 16917984  7124105 21889929
  14239911]]
[[0.80218154 0.88134098 0.89245576 0.9518522  0.96299958 1.02013934
  1.04897094 1.05241311 1.05501747]]


In [66]:
df_correct_preds.filter(pl.col("index") == idx)

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
4495,4,27012251,2019-03-18 07:00:43,"""Client_066""",4,1,1.356092,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,18,7,0,false,2.694183,1.938658,1.604773,1.429442,1.326386,-0.669069,-0.284955,-0.112912,0.0,0.0,1.037429,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,1.215413,-0.000208,1.455364,0.000056,0.71708,1.365456,0.513469,-1.287407,-1.409974,1.270989,0.147873,1.40623,-3.564944,-3.494551,-4.145818,-0.87125,-6.544776,-5.251174,-7.201351,0

In [67]:
df_train[indices.flatten().tolist()]

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
16917465,4,17270050,2019-02-18 07:03:44,"""Client_126""",5,1,0.359306,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,18,7,3,false,1.940574,1.374443,1.140296,1.02508,0.963055,-0.669069,-0.284955,-0.112912,0.0,0.0,0.844481,-0.917766,-0.698787,-0.576665,-0.484611,2.368202,-0.183119,-0.071133,0.0,0.0,0.748722,-0.000208,1.221308,-1.916514,0.71708,1.365456,0.974463,-1.043707,-1.409974,1.270989,-0.293992,1.383074,2.701878,2.68508,3.145725,0.61144,-4.456394,-4.815833,-6.692706,0

In [68]:
print(z_train[indices.flatten().tolist()][2:4])

[[-2.5287163   0.08925657 -1.0436959  -2.2916865   0.01805495  0.16156557
   0.12177697  0.02986081 -3.2514107   0.21756345 -1.7563679   1.2222317
  -0.18035884 -0.14375179 -2.9769034  -0.30376405]
 [-3.1217947   0.4865284  -1.1972506  -2.4588223   0.3951564  -0.55916935
   0.11703131 -0.30033958 -3.7261515  -0.10053436 -1.8222938   1.3768468
  -0.25881064  0.13256116 -2.8868325  -0.15595038]]


In [69]:
print(z_test[idx])

[-2.8849626   0.02349877 -1.0501988  -2.4290168  -0.09516186 -0.2293686
  0.26867017 -0.07365832 -3.6037285   0.06307574 -1.6560655   1.4680604
 -0.34788698  0.24830647 -3.1803067  -0.22151987]


### 2.4.2. Incorrect prediction:

In [74]:
df_wrong_preds.filter((pl.col("Model Prediction") == 4) & (pl.col("Anomaly") == 5))

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
37884,4,27267045,2019-03-18 08:00:30,"""Client_269""",5,1,-1.715342,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,18,8,0,false,1.864027,1.561209,1.612402,1.436114,1.332398,-0.669069,-0.284955,-0.112912,0.0,0.0,0.252484,0.695178,1.064677,1.987302,2.285504,-0.433523,-0.183119,-0.071133,0.0,0.0,-0.969538,-0.000208,-1.264866,0.000056,0.71708,1.365456,0.513469,-1.287407,-1.364252,0.64142,0.147873,1.40623,0.487243,0.334534,0.37224,-0.928549,0.133046,-2.262572,-3.709519,0.2039

In [86]:
idx = 2761298
x = torch.tensor(X_test[idx], dtype=torch.float32)
x[:6]

tensor([0.3377, 0.0000, 1.0000, 0.0000, 1.0000, 0.0000])

In [87]:
model.encoder.eval()
test_transaction_latent = model.encoder(x)
test_transaction_latent = test_transaction_latent.cpu().detach().numpy()
test_transaction_latent = test_transaction_latent.reshape(1, -1)
test_transaction_latent.shape

(1, 16)

In [88]:
distances, indices = knn.kneighbors(test_transaction_latent, n_neighbors=k)
print(indices)
print(distances)

[[17294684 15579676 15576175 15576935 15577749  1503830  1504681 15575782
   7525276]]
[[0.51974666 0.58682841 0.6071142  0.62124211 0.62175608 0.64193046
  0.64288425 0.64615911 0.69354057]]


In [89]:
df_wrong_preds.filter(pl.col("index") == idx)

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2761298,4,29315140,2019-03-25 20:55:25,"""Client_066""",5,1,0.337715,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,25,20,55,false,2.722973,1.960407,1.622689,1.445052,1.340441,-0.669069,-0.284955,-0.112912,0.0,0.0,1.042085,1.371203,-0.698787,-0.576665,-0.484611,2.420728,-0.183119,-0.071133,0.0,0.0,-0.242517,-0.000208,-1.474928,-3.567092,0.71708,1.365456,1.493561,0.276227,1.227968,1.900558,0.831314,1.143892,-0.18864,-0.742294,-0.89343,-0.803624,0.019852,1.160962,0.543246,

In [91]:
df_train[indices.flatten().tolist()]

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
17294684,4,17131041,2019-02-18 19:44:45,"""Client_066""",5,1,-0.612598,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,1,18,19,44,false,2.579309,1.851984,1.53335,1.367234,1.270427,-0.669069,-0.284955,-0.112912,0.0,0.0,0.578532,-0.917766,-0.698787,-0.576665,-0.484611,2.369213,-0.183119,-0.071133,0.0,0.0,-0.946471,-0.000208,-1.215285,-2.509273,0.71708,1.365456,0.974463,-1.043707,1.27369,1.270989,1.38338,-0.294236,-0.969714,-0.180879,-0.207235,0.392444,-0.189493,0.751603,-0.07

In [92]:
print(z_train[indices.flatten().tolist()][:2])

[[-2.4331093   0.9509997  -0.87176096 -3.1386654  -1.0228022  -0.88607335
  -0.27441552 -0.5133108  -3.018287   -0.18518315 -0.15477887  0.3923129
  -1.2918235   2.049067   -1.7767009  -0.7480476 ]
 [-2.4637065   0.84158844 -0.9778522  -2.8729079  -0.81690234 -0.7319219
  -0.25929818 -0.4418549  -3.2682087   0.06021937  0.24896924  0.52185583
  -1.7033967   1.9127183  -1.7117262  -0.578028  ]]


In [93]:
print(z_test[idx])

[-2.450901    0.9282147  -0.891068   -3.0944896  -0.90527314 -0.7225768
 -0.4362544  -0.4227226  -3.0839684  -0.02157213  0.04562509  0.56950283
 -1.4937688   2.0946276  -1.5747468  -0.82387644]


## 2.5. An unusually large amount of collateral transferred in and out of an account within a short period of time:
This behavior is unusual as a client would not be able to invest by simply trading collateral, or at least such a strategy would be unusual.  
Selected indexes from testing set: 2762376, 516444

### 2.5.1. Correct prediction:

In [94]:
df_correct_preds.filter((pl.col("Anomaly") == 5))

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
37661,5,27267059,2019-03-18 08:00:06,"""Client_269""",5,1,-0.056761,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,18,8,0,false,2.706488,1.947968,1.612474,1.436145,1.332426,-0.669069,-0.284955,-0.112912,0.0,0.0,1.097752,1.392918,1.733295,2.015149,-0.484611,2.410997,-0.183119,-0.071133,0.0,0.0,-0.984395,-0.000208,1.113591,-2.277635,0.71708,1.365456,0.513469,-1.287407,-1.364252,0.64142,0.147873,1.40623,0.540915,0.35138,0.392118,-0.925693,0.136732,-2.270245,-3.718484,0.204,

In [100]:
idx = 2762376
x = torch.tensor(X_test[idx], dtype=torch.float32)
x[:6]

tensor([0.3138, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000])

In [101]:
model.encoder.eval()
test_transaction_latent = model.encoder(x)
test_transaction_latent = test_transaction_latent.cpu().detach().numpy()
test_transaction_latent = test_transaction_latent.reshape(1, -1)
test_transaction_latent.shape

(1, 16)

In [102]:
distances, indices = knn.kneighbors(test_transaction_latent, n_neighbors=k)
print(indices)
print(distances)

[[19623124  7598978 14740996 19287387 20164346  5526145 19624980 19288534
  19739580]]
[[0.52873516 0.55274493 0.56253332 0.56486255 0.57306278 0.58062702
  0.5827989  0.58416122 0.58481294]]


In [103]:
df_correct_preds.filter(pl.col("index") == idx)

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2762376,5,29315121,2019-03-25 20:57:30,"""Client_066""",5,1,0.313846,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,25,20,57,false,2.722976,1.960468,1.622718,1.445076,1.340454,-0.669069,-0.284955,-0.112912,0.0,0.0,1.04478,-0.917766,-0.698787,-0.576665,-0.484611,2.420729,-0.183119,-0.071133,0.0,0.0,0.742409,-0.000208,-1.474181,-2.09058,0.71708,1.365456,1.493561,0.276227,1.227968,1.900558,0.575268,1.291715,-0.404574,-0.763154,-0.902047,-0.807294,-0.01851,1.159393,0.545971,

In [104]:
df_train[indices.flatten().tolist()]

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
19623124,5,19675081,2019-02-25 14:53:05,"""Client_066""",5,1,-0.425004,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,25,14,53,false,2.615835,1.879581,1.556082,1.387042,1.288228,-0.669069,-0.284955,-0.112912,0.0,0.0,1.008023,-0.917766,-0.698787,-0.576665,-0.484611,2.382558,-0.183119,-0.071133,0.0,0.0,-0.825706,-0.000208,1.014934,-3.22835,0.71708,1.365456,1.199668,0.943877,0.279149,-1.078596,1.051029,0.946066,-0.038907,-0.667843,-0.869572,0.055654,0.324298,0.515895,-0.454

In [105]:
print(z_train[indices.flatten().tolist()][0])

[-1.7206421  -0.84009975 -0.9258529  -1.1255988  -0.8609636   1.2533833
  1.1971354  -1.6296676  -1.4845101  -0.5260371   0.6082765  -0.27374735
 -1.2463502   2.2190332  -1.8727342   2.6298902 ]


In [106]:
print(z_test[idx])

[-1.7510817  -0.6094156  -0.88196003 -1.2582811  -0.91334903  0.9356589
  1.2273142  -1.8647422  -1.5614073  -0.54726833  0.5948409  -0.24358319
 -1.3451564   2.230141   -1.9320678   2.4712605 ]


### 2.5.2. Incorrect prediction:

In [117]:
df_wrong_preds.filter((pl.col("Model Prediction") == 5))

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1197,5,27080512,2019-03-18 02:06:54,"""Client_060""",3,1,0.406096,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,1,18,2,6,false,1.832085,1.300184,1.084277,1.269644,1.202732,-0.669069,-0.284955,-0.112912,0.0,0.0,1.036357,-0.917766,-0.698787,-0.576665,-0.484611,2.410615,-0.183119,-0.071133,0.0,0.0,1.027628,-0.000208,0.927427,-3.113357,0.71708,1.365456,0.513469,-1.287407,-0.415434,3.620574,-0.707075,1.224512,-8.540134,-3.662876,-4.344431,-0.885931,-7.798274,-7.042817,-9.29467

In [139]:
idx = 516444
x = torch.tensor(X_test[idx], dtype=torch.float32)
x[:6]

tensor([0.6409, 0.0000, 1.0000, 1.0000, 0.0000, 0.0000])

In [140]:
model.encoder.eval()
test_transaction_latent = model.encoder(x)
test_transaction_latent = test_transaction_latent.cpu().detach().numpy()
test_transaction_latent = test_transaction_latent.reshape(1, -1)
test_transaction_latent.shape

(1, 16)

In [141]:
distances, indices = knn.kneighbors(test_transaction_latent, n_neighbors=k)
print(indices)
print(distances)

[[13367421   112867  2231618 19474445 15410589 23586083   115285 13258223
    114612]]
[[0.7462011  0.8666153  0.97474718 0.98308575 0.99158931 1.01107335
  1.03264058 1.05734622 1.0585593 ]]


In [142]:
df_wrong_preds.filter(pl.col("index") == idx)

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
516444,5,27528470,2019-03-19 08:52:05,"""Client_066""",4,1,0.640881,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,2,19,8,52,false,2.698378,1.94185,1.607398,1.431725,1.328446,-0.669069,-0.284955,-0.112912,0.0,0.0,-1.390533,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,0.820922,-0.000208,-0.000167,0.000056,-0.811648,0.879612,0.777923,-1.174554,-1.364252,0.64142,1.144188,0.831028,-0.12155,-1.364005,-0.636134,-1.079593,-0.393374,-1.785576,0.165

In [143]:
df_train[indices.flatten().tolist()]

index,Model Prediction,id,datetime,Originator,Anomaly,Anomaly_bin,amount_log,InputOutput_Buy,InputOutput_Sell,Market_1,Market_2,Market_3,Market_4,Product Type_ADR,Product Type_Bond,Product Type_CAADR,Product Type_ETOEquity,Product Type_ETOEquityIndex,Product Type_Equity,Product Type_FX,Product Type_FXForward,Product Type_FXSwap,Product Type_FutureBond,Product Type_FutureCommodity,Product Type_FutureEquity,Product Type_FutureEquityIndex,Product Type_FutureFX,Product Type_FutureOptionEquityIndex,Product Type_Repo,Product Type_SimpleTransfer,Product Class_ADR Conversion,"Product Class_Cash in / out (withdrawal), Security in / out",Product Class_External fee,Product Class_Trade,Currency_1,Currency_2,day_of_week,day_of_month,hour,minute,is_rounded,originator_index_timestamp_diff_1,originator_index_timestamp_diff_2,originator_index_timestamp_diff_3,originator_index_timestamp_diff_4,originator_index_timestamp_diff_5,market_index_timestamp_diff_1,market_index_timestamp_diff_2,market_index_timestamp_diff_3,market_index_timestamp_diff_4,market_index_timestamp_diff_5,product_type_index_timestamp_diff_1,product_type_index_timestamp_diff_2,product_type_index_timestamp_diff_3,product_type_index_timestamp_diff_4,product_type_index_timestamp_diff_5,product_class_index_timestamp_diff_1,product_class_index_timestamp_diff_2,product_class_index_timestamp_diff_3,product_class_index_timestamp_diff_4,product_class_index_timestamp_diff_5,originator_index_amount_diff_1,market_index_amount_diff_1,product_type_index_amount_diff_1,product_class_index_amount_diff_1,day_of_week_sin,day_of_week_cos,day_of_month_sin,day_of_month_cos,hour_sin,hour_cos,minute_sin,minute_cos,rolling_mean_1h,rolling_mean_12h,rolling_mean_24h,rolling_mean_7d,rolling_sum_1h,rolling_sum_12h,rolling_sum_24h,rolling_sum_7d,rolling_std_1h,rolling_std_12h,rolling_std_24h,rolling_std_7d,rolling_max_1h,rolling_max_12h,rolling_max_24h,rolling_max_7d,rolling_min_1h,rolling_min_12h,rolling_min_24h,rolling_min_7d,rolling_n_transactions_1h,rolling_n_transactions_12h,rolling_n_transactions_24h,rolling_n_transactions_7d,rolling_mean_originator_1h,rolling_mean_originator_12h,rolling_mean_originator_24h,rolling_mean_originator_7d,rolling_std_originator_1h,rolling_std_originator_12h,rolling_std_originator_24h,rolling_std_originator_7d,rolling_max_originator_1h,rolling_max_originator_12h,rolling_max_originator_24h,rolling_max_originator_7d,rolling_min_originator_1h,rolling_min_originator_12h,rolling_min_originator_24h,rolling_min_originator_7d,rolling_sum_originator_1h,rolling_sum_originator_12h,rolling_sum_originator_24h,rolling_sum_originator_7d,rolling_n_transactions_originator_1h,rolling_n_transactions_originator_12h,rolling_n_transactions_originator_24h,rolling_n_transactions_originator_7d,cumcount_InputOutput,cumcount_Market,cumcount_Product Type,cumcount_Product Class,cumcount_Currency,InputOutput_freq_originator,Market_freq_originator,Product Type_freq_originator,Product Class_freq_originator,Currency_freq_originator
u32,i64,u32,datetime[μs],str,i64,i8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,i8,i8,i8,i8,bool,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
13367421,5,13546615,2019-02-07 10:53:00,"""Client_212""",5,1,0.493698,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,4,7,10,53,false,2.109769,1.509972,1.251864,1.123152,1.051033,-0.669069,-0.284955,-0.112912,0.0,0.0,-1.390533,-0.917766,-0.698787,-0.576665,-0.484611,-0.433523,-0.183119,-0.071133,0.0,0.0,-1.124702,-0.000208,-0.000167,0.000056,-0.1313,-1.087529,-1.298914,0.376776,-1.016961,-0.449026,1.051029,0.946066,-1.012188,-0.921288,-0.43735,-0.345983,0.188849,-0.447423,0

In [144]:
print(z_train[indices.flatten().tolist()][:3])

[[-2.2361984   0.5273805  -1.1199679  -1.6991603  -1.2989526   0.03733809
  -0.4013431   0.24374136 -3.533218    0.66975945  0.6517832   1.0006684
  -1.4776965   1.9888917  -2.3299892  -0.5722429 ]
 [-2.3451464   0.45152074 -1.120382   -1.9146601  -1.0101213  -0.05800132
  -0.51240224  0.495614   -3.5361      0.71850234  0.7863638   0.74018717
  -1.4431381   2.2859602  -2.1540463   0.01843518]
 [-2.4961991   0.21570994 -0.99202037 -2.102909   -1.0669441  -0.18767384
  -0.23542306  0.33791563 -3.3748226   0.52187365  0.5995053   0.75173664
  -1.5977088   1.8852743  -2.1540031  -0.19988418]]
